In [1]:
import cv2
import torch
import os
from PIL import Image

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

In [2]:
model = torch.hub.load('ultralytics/yolov5', 'custom', '../models/yoloFire.pt')  # force_reload=True to update

# Other Parameters
pd = 0
text = "emergency"
font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 0.8
font_thickness = 1
text_color = (255, 255, 255)  # White color
pixel_length_meters = 0.05

Using cache found in C:\Users\vikas/.cache\torch\hub\ultralytics_yolov5_master
C:\Users\vikas\Desktop\ForestFireTracking\envfire\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
YOLOv5  2024-5-25 Python-3.10.4 torch-2.2.1+cpu CPU

Fusing layers... 
custom_YOLOv5s summary: 232 layers, 7246518 parameters, 0 gradients
Adding AutoShape... 


In [7]:
def yolo(im, yolo_frame, size=640):
    g = (size / max(im.size))  
    im = im.resize((int(x * g) for x in im.size), Image.LANCZOS)
    results = model(im)
    
    result_image = Image.fromarray(results.ims[0])
    result_frame = np.array(result_image)
    
#     results.render()  -> YOLO Inference

    for box in results.xyxy[0]:
        xmin, ymin, xmax, ymax, _, _ = box
        cv2.rectangle(result_frame, (int(xmin), int(ymin)), (int(xmax), int(ymax)), (255, 0, 0), 2)
        cv2.rectangle(yolo_frame, (int(xmin), int(ymin)), (int(xmax), int(ymax)), (0, 0, 255), -1)
        
                
    area = 0
    gray = cv2.cvtColor(yolo_frame, cv2.COLOR_BGR2GRAY)
    ret, thresh = cv2.threshold(gray, 50, 255, cv2.THRESH_BINARY)

    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Iterate through each contour
    for contour in contours:
        # Calculate area of the contour
        pixel_area = cv2.contourArea(contour)  # Area in pixels

        area += pixel_area
            # Draw contour on the original image
#             cv2.drawContours(black_frame, [contour], -1, (0, 255, 0), 1)
        
#         pr = area*100/(black_frame.shape[0]*black_frame.shape[1])
        
#         cv2.putText(black_frame, "Area: {}%".format(round(pr,2)), (20, 370), font, font_scale, text_color, font_thickness, lineType=cv2.LINE_AA)
    
#     return black_frame

    

    return result_frame, yolo_frame, results, area

In [8]:
def motion(mtion_frame, prvs, new):
    # Calculate Optical Flow
    flow = cv2.calcOpticalFlowFarneback(prvs, new, None, 
                                        0.5, 3, 15, 3, 5, 1.2, 0)

    # Overlay motion visualization onto the original frame
    for y in range(0, motion_frame.shape[0], 10):
        for x in range(0, motion_frame.shape[1], 10):
            fx, fy = flow[y, x]
            cv2.line(motion_frame, (x, y), (int(x + fx), int(y + fy)), [0,0,255], 1)

    return flow, motion_frame

In [9]:
start_frame = 1000  # Change this to the frame number you want to start from

cap = cv2.VideoCapture('../demoVideos/paperFire1.mp4')
# cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

_, old_frame = cap.read()

#### variables for yolo #####
yolo_area_frame = np.zeros_like(old_frame)
old_area = 0
areas = []
area_growth = []

#### variables for motion #####
# -> -> - > Clockwise positive direction
motion_frame = np.zeros_like(old_frame)
prvs = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
avgx = []
avgy = []
angles = []
speed = []

while True:
    # Read a frame from the video
    ret, new_frame = cap.read()
    
    # Break the loop if the video is over
    if not ret:
        print('Error in capturing frames')
        break
    
    # Convert the frame from OpenCV format (BGR) to PIL format (RGB)
    frame_pil = Image.fromarray(cv2.cvtColor(new_frame, cv2.COLOR_BGR2RGB))
    
    # Area analysis using YOLO
    yolo_result_frame, yolo_frame, yolo_results, new_area = yolo(frame_pil, yolo_area_frame)
    areas.append(new_area)
    area_growth.append(new_area-old_area)
    old_area = new_area
    
    cv2.imshow('YOLOv5 result', yolo_result_frame)
    

    # Motion analysis using Farneback
    new = cv2.cvtColor(new_frame, cv2.COLOR_BGR2GRAY)
    
    flow, motion_frame = motion(motion_frame, prvs, new)
    
    cv2.imshow('Motion analysis', motion_frame)
    
    prvs = new
    
    # Get average flow direction
    avgx.append(np.mean(flow[..., 0]))
    avgy.append(np.mean(flow[..., 1]))


    if len(avgx) > 30: 
        avg_fx = np.mean(avgx)
        avg_fy = np.mean(avgy)
        avg_direction_angle = np.arctan2(avg_fy, avg_fx)
        avg_direction_degrees = np.degrees(avg_direction_angle)
        
        avg_speed = np.sqrt(avg_fx**2 + avg_fy**2)
        
        speed.append(avg_speed)
        
        angles.append(avg_direction_degrees)

        # Set text on black_frame
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(new_frame, f"Direction wrt X+: {avg_direction_degrees:.2f} degrees", (10, 30), font, 1, (0, 255, 0), 1, cv2.LINE_AA)

        avgx.pop()
        avgy.pop()
        
    cv2.imshow('Frame', new_frame)
    
    # Press q to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Error in capturing frames


In [10]:
cap.release()
cv2.destroyAllWindows()

In [11]:
# Get min and max values for each variable
min_area = min(areas)
max_area = max(areas)
min_growth = min(area_growth)
max_growth = max(area_growth)
min_speed = min(speed)
max_speed = max(speed)
min_angle = min(angles)
max_angle = max(angles)


### Code to plot metrics

In [12]:
from sklearn.preprocessing import MinMaxScaler

# area_scaler = MinMaxScaler()
# areas_normalized = area_scaler.fit_transform(np.array(areas).reshape(-1, 1)).flatten()

# growth_scaler = MinMaxScaler()
# area_growth_normalized = growth_scaler.fit_transform(np.array(area_growth).reshape(-1, 1)).flatten()

# speed_scaler = MinMaxScaler()
# speed_normalized = speed_scaler.fit_transform(np.array(speed).reshape(-1, 1)).flatten()

cap = cv2.VideoCapture('../demoVideos/paperFire1.mp4')
fps = cap.get(cv2.CAP_PROP_FPS)

# Generate the x-axis values in seconds
frame_numbers = np.arange(len(areas))  # Assuming areas, area_growth, speed all have the same length
time_seconds = frame_numbers / fps

# Plotting
plt.figure(figsize=(12, 8))


# Plot for areas
plt.subplot(2, 2, 1)
sns.lineplot(x=time_seconds, y=areas, color='b', linewidth=1)
plt.ylabel('Area (pixels)', fontsize=12)
plt.xlabel('Time (seconds)', fontsize=12)
plt.title(f'Area (Min: {min(areas):.2f}, Max: {max(areas):.2f}, Dev: {np.std(areas):.2f})')

# Plot for area growth
plt.subplot(2, 2, 3)
sns.lineplot(x=time_seconds, y=area_growth, color='r', linewidth=1)
plt.ylabel('Growth (pixels)', fontsize=12)
plt.xlabel('Time (seconds)', fontsize=12)
plt.title(f'Growth (Min: {min(area_growth):.2f}, Max: {max(area_growth):.2f}, Dev: {np.std(area_growth):.2f})')

# Plot for speed
plt.subplot(2, 2, 2)
sns.lineplot(x=time_seconds[:632], y=speed, color='g', linewidth=1)
plt.ylabel('Speed (pixels)', fontsize=12)
plt.xlabel('Time (seconds)', fontsize=12)
plt.title(f'Speed (Min: {min(speed):.2f}, Max: {max(speed):.2f}, Dev: {np.std(speed):.2f})')

# Plotting polar plot in the second column
plt.subplot(2, 2, 4, polar=True)
plt.hist(np.radians(angles), bins=30, color='skyblue', alpha=0.7)
plt.xlabel('Direction', fontsize=12)
plt.gca().set_theta_direction(-1)
plt.gca().set_rticks([])
plt.title('Direction')

# Optionally, adjust the spacing between subplots
plt.tight_layout()
plt.subplots_adjust(hspace=0.4, wspace=0.4)

plt.savefig('fire_metrics.png')

In [15]:
len(areas)

662

In [16]:
len(angles)

632

In [17]:
len(time_seconds)

662